In [ ]:
pip install djitellopy opencv-python numpy

In [ ]:
from djitellopy import Tello
import cv2
import numpy as np
import time
import logging

# Suppress INFO and WARNING messages from djitellopy library
logging.getLogger('djitellopy').setLevel(logging.ERROR)

#####################################
# Initialization of coordinates
#####################################
# Coordinate system:
#   x: positive = forward, negative = backward
#   y: positive = left, negative = right
#   z: positive = up, negative = down
# Initial starting position of ego drone (at the center of the picnic blanket)
ego_x = 0
ego_y = 0
ego_z = 0

# State history: stores full state (x, y, z) at each time step
state_history = []  # List of tuples: [(x, y, z), ...]

#####################################
# Define borders
#####################################
# Define bounding box around initial starting point (200 x 200)
# Box extends 100 units in each direction from center
# BOX_SIZE = 200 # random estimate rn for the bounding box of the ODD, will test based on picnic blanket
# BOX_HALF = BOX_SIZE / 2  # 100 units
# box_x_min = ego_x - BOX_HALF
# box_x_max = ego_x + BOX_HALF
# box_y_min = ego_y - BOX_HALF
# box_y_max = ego_y + BOX_HALF

#####################################
# Adversary position (stationary)
#####################################
adversary_active = True  # Set to True to enable adversary avoidance

# Adversary position (will be set after takeoff to same z-height)
adversary_x = 40 #20  # Fixed x position of adversary
adversary_y = 0   # Fixed y position of adversary
adversary_z = None  # Will be set to takeoff height after takeoff

# Threshold distance for collision avoidance (squared distance: x^2 + y^2)
avoidance_radius = 10 #30  # 10 #cm threshold (reduced from 20 for looser avoidance)
adversary_threshold = avoidance_radius**2  # 10^2 = 100 (10 cm threshold)

# CBF parameters
CBF_ALPHA = 0.5 # CBF gain parameter (h_dot >= -alpha * h)

#####################################
# Goal tracking
#####################################
goal_index = 0
goals = [
    # CBF Test Scenario: Adversary is at (40, 0) - 40 cm forward (in front of start)
    # Coordinate system: x = forward/backward, y = left/right
    # Start at (0, 0), adversary at (40, 0) means 40 cm forward
    
    #(20, 0),   
    (40, -40),  
    (0, -40),
    (0, -20)     
    
    # Alternative scenario - go around the other side (uncomment to try):
    # (80, 0),   # Goal 1: Behind adversary
    # (80, -40), # Goal 2: Behind and 40 cm right
    # (40, -40), # Goal 3: Beside adversary on right
    # (0, -40),  # Goal 4: Return path to the right
    # (0, 0)     # Goal 5: Back to start
]

'''
Drone building
'''
#####################################
# Initialization of motion params
#####################################
upward_initialization = 20 #20 cm is the minimum
move_step = 20#40

#####################################
# Initialization of drone
#####################################
tello = Tello()
#print("Connecting to Tello...")
tello.connect()
print(f"Battery level: {tello.get_battery()}%")
tello.streamon()

print("Press 't' to takeoff, 'q' to land and quit.")
drone_in_air = False

#####################################
# Definitions
#####################################
def compute_CBF(current_x, current_y, desired_x, desired_y, adversary_x, adversary_y):
    # Relative position vector from adversary to current position
    rel_x = current_x - adversary_x
    rel_y = current_y - adversary_y
    
    # Safety margin function h = d^2 - r^2
    # h > 0: Safe, h = 0: On boundary, h < 0: Unsafe
    h = rel_x**2 + rel_y**2 - avoidance_radius**2
    
    # Compute the desired change in position (discrete "velocity")
    desired_dx = desired_x - current_x
    desired_dy = desired_y - current_y
    
    # Approximate derivative of h (using discrete step as velocity)
    # h_dot = 2 * (rel_x * dx + rel_y * dy)
    h_dot = 2 * (rel_x * desired_dx + rel_y * desired_dy)
    
    # CBF safety condition: h_dot >= -alpha * h
    # If this condition is satisfied, the desired movement is safe
    if h_dot >= -CBF_ALPHA * h:
        return desired_x, desired_y  # Desired movement is safe, return as-is
    
    # --- Desired movement is NOT safe ---
    # The desired movement violates the CBF condition.
    # Instead of blocking it, we modify it to be tangent to the safety circle.
    
    # Compute distance to adversary
    dist = np.sqrt(rel_x**2 + rel_y**2)
    
    if dist == 0:
        # On top of adversary, move away
        return current_x, current_y  # Stay put or could move away, but for safety stay put
    
    # Normalized normal vector (pointing from adversary to current position)
    norm_x = rel_x / dist
    norm_y = rel_y / dist
    
    # Project desired movement onto the normal vector
    # This gives the scalar magnitude of movement in the normal direction
    movement_normal_scalar = desired_dx * norm_x + desired_dy * norm_y
    
    # movement_normal_scalar > 0 means moving away (outwards)
    # movement_normal_scalar < 0 means moving towards (inwards) - this is unsafe
    
    # Calculate the purely tangential component of the movement
    # movement_tangent = movement_desired - movement_normal
    movement_tangent_x = desired_dx - (movement_normal_scalar * norm_x)
    movement_tangent_y = desired_dy - (movement_normal_scalar * norm_y)
    
    # Check if tangential component is too small (happens when moving directly towards/away from adversary)
    tangent_magnitude = np.sqrt(movement_tangent_x**2 + movement_tangent_y**2)
    
    if tangent_magnitude < 0.1:  # Tangential component is essentially zero
        # When moving directly towards adversary, we need to add a perpendicular component
        # to allow navigation around the obstacle. Use a direction perpendicular to the normal.
        # Perpendicular vector: rotate normal by 90 degrees: (norm_y, -norm_x) or (-norm_y, norm_x)
        
        # Calculate goal direction from current position
        goal_dx = desired_x - current_x
        goal_dy = desired_y - current_y
        
        # Choose perpendicular direction that helps reach the goal
        # Check which perpendicular direction is closer to the goal direction
        perp1_x = norm_y
        perp1_y = -norm_x
        perp2_x = -norm_y
        perp2_y = norm_x
        
        # Dot product with goal direction to choose best perpendicular
        dot1 = goal_dx * perp1_x + goal_dy * perp1_y
        dot2 = goal_dx * perp2_x + goal_dy * perp2_y
        
        # Use the perpendicular direction that aligns better with goal direction
        # If equal, prefer the one with positive dot product (helps reach goal)
        if abs(dot1) > abs(dot2) or (abs(dot1) == abs(dot2) and dot1 > 0):
            perp_x, perp_y = perp1_x, perp1_y
        else:
            perp_x, perp_y = perp2_x, perp2_y
        
        # Scale perpendicular to move_step to ensure meaningful movement
        movement_tangent_x = perp_x * move_step
        movement_tangent_y = perp_y * move_step
    
    # This tangential movement is orthogonal to the pursuer direction.
    # Its h_dot will be 0, which satisfies: 0 >= -CBF_ALPHA * h (as long as h >= 0)
    # So this movement is guaranteed to be safe.
    
    # Compute safe position by adding tangential movement to current position
    safe_x = current_x + movement_tangent_x
    safe_y = current_y + movement_tangent_y
    
    # Debug: Print when CBF modifies movement
    print(f"  [CBF ACTIVE] Modified movement: desired ({desired_x:.1f}, {desired_y:.1f}) -> safe ({safe_x:.1f}, {safe_y:.1f})")
    print(f"    Distance to adversary: {dist:.1f} cm, Safety margin h: {h:.1f}")
    
    return safe_x, safe_y

def is_goal_blocked(goal_x, goal_y, adversary_x, adversary_y):
    dist_to_adversary_sq = (goal_x - adversary_x)**2 + (goal_y - adversary_y)**2
    return dist_to_adversary_sq < adversary_threshold






'''
Takeoff
'''
#####################################
# Drone maneuverance after takeoff
#####################################
try:
    # Record initial state before loop starts
    state_history.append((ego_x, ego_y, ego_z))
    
    while True:
        frame = tello.get_frame_read().frame
        frame = cv2.resize(frame, (720, 480))
        h, w, _ = frame.shape

        # Display the frame
        cv2.imshow("Tello Tracking Feed", frame)

        key = cv2.waitKey(1) & 0xFF

        # Controls for drone
        if key == ord('t') and not drone_in_air:
            tello.takeoff()
            drone_in_air = True
            print("Drone is now airborne and hovering.")
            tello.move_up(upward_initialization)  # move up 30 cm after takeoff
            ego_z += upward_initialization
            adversary_z = ego_z  # Set pursuer z to same height as takeoff height
            state_history.append((ego_x, ego_y, ego_z))  # Record state after movement
            print(f"Drone moved up 30 cm. Current state: ({ego_x}, {ego_y}, {ego_z})")
            print(f"Pursuer set at position: ({adversary_x}, {adversary_y}, {adversary_z})")
            if adversary_active:
                print(f"Adversary detected at ({adversary_x}, {adversary_y}), avoidance radius: {avoidance_radius} cm")

        # if key == ord('t') and not drone_in_air:
        #     tello.takeoff()
        #     time.sleep(2)  # Delay after takeoff to allow drone to stabilize
        #     drone_in_air = True
        #     print("Drone is now airborne.")
            
        #     # Gradual takeoff using velocity control
        #     target_height = 30  # cm you want to reach (minimum safe height)
        #     hover_speed = 10    # cm/s (upward velocity)
        #     steps = int(target_height / hover_speed)
            
        #     for _ in range(steps):
        #         tello.send_rc_control(0, 0, hover_speed, 0)  # (left-right, forward-back, up-down, yaw)
        #         ego_z += hover_speed
        #         state_history.append((ego_x, ego_y, ego_z))
        #         time.sleep(1)  # sleep 1 second per step

        #     # Stop upward motion and hover
        #     #tello.send_rc_control(0, 0, 0, 0)
        #     time.sleep(0.5)  # Delay after stopping motion
        #     adversary_z = ego_z
        #     print(f"Reached hover height: {ego_z} cm")
        #     if adversary_active:
        #         print(f"Adversary detected at ({adversary_x}, {adversary_y}), avoidance radius: {avoidance_radius} cm")

        


        # drone movement based on goals
        if drone_in_air and goal_index < len(goals):
            target_x, target_y = goals[goal_index]
            
            # Check if goal is blocked by adversary (switching-based control)
            if adversary_active and is_goal_blocked(target_x, target_y, adversary_x, adversary_y):
                print(f"Goal {goal_index + 1} at ({target_x}, {target_y}) is BLOCKED by adversary at ({adversary_x}, {adversary_y})")
                print(f"Skipping goal {goal_index + 1} and moving to next goal...")
                goal_index += 1
                time.sleep(0.5)
                
                # Check if we've processed all goals (no linked list - don't return to skipped goals)
                if goal_index >= len(goals):
                    print("All goals processed! Landing the drone...")
                    if drone_in_air:
                        tello.land()
                        time.sleep(3)  # Delay after landing
                        state_history.append((ego_x, ego_y, ego_z))  # Record final state
                        print("Drone has landed.")
                        print(f"Final state: ({ego_x}, {ego_y}, {ego_z})")
                        print(f"Total states recorded: {len(state_history)}")
                    break
                continue
            
            # Calculate distance to current goal
            dx = target_x - ego_x
            dy = target_y - ego_y
            distance_to_goal = np.sqrt(dx**2 + dy**2)
            
            # Threshold for considering goal reached (within 10 cm)
            goal_threshold = 10
            




            if distance_to_goal > goal_threshold:
                # True CBF Controller: Always compute a safe movement
                # Step 1: Calculate desired position (one step towards goal)
                # Check if each dimension is already within threshold
                x_within_threshold = abs(dx) < goal_threshold
                y_within_threshold = abs(dy) < goal_threshold
                
                desired_x = ego_x
                desired_y = ego_y
                
                # If one dimension is already within threshold, only move in the other dimension
                if x_within_threshold and not y_within_threshold:
                    # X is close enough, only move in Y direction
                    if dy > 0:
                        desired_y = ego_y + move_step
                    else:
                        desired_y = ego_y - move_step
                elif y_within_threshold and not x_within_threshold:
                    # Y is close enough, only move in X direction
                    if dx > 0:
                        desired_x = ego_x + move_step
                    else:
                        desired_x = ego_x - move_step
                elif not x_within_threshold and not y_within_threshold:
                    # Both dimensions need movement, prioritize the direction with larger error
                    if abs(dx) >= abs(dy):
                        if dx > 0:
                            desired_x = ego_x + move_step
                        else:
                            desired_x = ego_x - move_step
                    else:
                        if dy > 0:
                            desired_y = ego_y + move_step
                        else:
                            desired_y = ego_y - move_step
                # If both are within threshold, desired_x and desired_y stay as ego_x and ego_y (no movement)
                
                # Step 2: Apply CBF to get safe position
                if adversary_active:
                    safe_x, safe_y = compute_CBF(ego_x, ego_y, desired_x, desired_y, adversary_x, adversary_y)
                else:
                    safe_x, safe_y = desired_x, desired_y
                
                # Step 3: Move to safe position
                safe_dx = safe_x - ego_x
                safe_dy = safe_y - ego_y
                
                # Normalize safe movement direction to move_step distance for discrete movement
                safe_dist = np.sqrt(safe_dx**2 + safe_dy**2)
                if safe_dist > 0.1:  # Only move if there's a meaningful movement
                    # Normalize to move_step
                    if safe_dist > move_step:
                        safe_dx = (safe_dx / safe_dist) * move_step
                        safe_dy = (safe_dy / safe_dist) * move_step
                    # If safe_dist < move_step, use the actual safe movement (smaller step)
                    
                    # Execute movement based on safe position
                    # Convert to integers for Tello commands
                    # Tello requires minimum 20 cm, maximum 500 cm movements for discrete commands
                    # For smaller movements, use RC velocity control
                    MIN_MOVE = 20
                    MAX_MOVE = 40  # Set to 40 for safety
                    SMALL_VELOCITY = 4  # cm/s for small movements using RC control
                    
                    movement_occurred = False
                    if abs(safe_dx) > 0.1:  # Small threshold to avoid tiny movements
                        move_x = abs(safe_dx)
                        original_move_x = move_x
                        
                        if move_x < MIN_MOVE:
                            # Use RC velocity control for small movements to maintain safety
                            # Calculate time needed: distance / velocity
                            move_time = move_x / SMALL_VELOCITY
                            # RC control: send_rc_control(left_right, forward_back, up_down, yaw)
                            # Values: -100 to 100, but we'll use a small value for slow movement
                            # For forward: positive forward_back, for backward: negative forward_back
                            rc_velocity = 10  # Small RC value (10) for ~4 cm/s movement
                            
                            if safe_dx > 0:
                                tello.send_rc_control(0, rc_velocity, 0, 0)  # Forward
                                print(f"  Using RC control: Moving forward {original_move_x:.1f} cm at {SMALL_VELOCITY} cm/s")
                            else:
                                tello.send_rc_control(0, -rc_velocity, 0, 0)  # Backward
                                print(f"  Using RC control: Moving back {original_move_x:.1f} cm at {SMALL_VELOCITY} cm/s")
                            
                            time.sleep(move_time)
                            tello.send_rc_control(0, 0, 0, 0)  # Stop
                            time.sleep(0.2)  # Brief pause after stopping
                            
                            if safe_dx > 0:
                                ego_x += original_move_x
                            else:
                                ego_x -= original_move_x
                            movement_occurred = True
                        elif move_x > MAX_MOVE:
                            # Clamp large movements
                            move_x = MAX_MOVE
                            print(f"  X movement clamped to {MAX_MOVE} cm for safety")
                            if safe_dx > 0:
                                tello.move_forward(int(move_x))
                                print(f"  Moved forward {move_x} cm!")
                                time.sleep(0.5)
                                ego_x += move_x
                                movement_occurred = True
                            else:
                                tello.move_back(int(move_x))
                                print(f"  Moved back {move_x} cm!")
                                time.sleep(0.5)
                                ego_x -= move_x
                                movement_occurred = True
                        else:
                            # Normal discrete movement
                            move_x = int(round(move_x))
                            if safe_dx > 0:
                                tello.move_forward(move_x)
                                print(f"  Moved forward {move_x} cm!")
                                time.sleep(0.5)
                                ego_x += move_x
                                movement_occurred = True
                            else:
                                tello.move_back(move_x)
                                print(f"  Moved back {move_x} cm!")
                                time.sleep(0.5)
                                ego_x -= move_x
                                movement_occurred = True
                    
                    if abs(safe_dy) > 0.1:  # Small threshold to avoid tiny movements
                        move_y = abs(safe_dy)
                        original_move_y = move_y
                        
                        if move_y < MIN_MOVE:
                            # Use RC velocity control for small movements to maintain safety
                            # Calculate time needed: distance / velocity
                            move_time = move_y / SMALL_VELOCITY
                            # RC control: send_rc_control(left_right, forward_back, up_down, yaw)
                            # Note: In RC control, NEGATIVE left_right = left, POSITIVE left_right = right
                            # But in our code: safe_dy > 0 = left, safe_dy < 0 = right
                            # So we need to flip the sign
                            rc_velocity = 10  # Small RC value (10) for ~4 cm/s movement
                            
                            if safe_dy > 0:
                                tello.send_rc_control(-rc_velocity, 0, 0, 0)  # Left (negative for left in RC)
                                print(f"  Using RC control: Moving left {original_move_y:.1f} cm at {SMALL_VELOCITY} cm/s")
                            else:
                                tello.send_rc_control(rc_velocity, 0, 0, 0)  # Right (positive for right in RC)
                                print(f"  Using RC control: Moving right {original_move_y:.1f} cm at {SMALL_VELOCITY} cm/s")
                            
                            time.sleep(move_time)
                            tello.send_rc_control(0, 0, 0, 0)  # Stop
                            time.sleep(0.2)  # Brief pause after stopping
                            
                            if safe_dy > 0:
                                ego_y += original_move_y
                            else:
                                ego_y -= original_move_y
                            movement_occurred = True
                        elif move_y > MAX_MOVE:
                            # Clamp large movements
                            move_y = MAX_MOVE
                            print(f"  Y movement clamped to {MAX_MOVE} cm for safety")
                            if safe_dy > 0:
                                tello.move_left(int(move_y))
                                print(f"  Moved left {move_y} cm!")
                                time.sleep(0.5)
                                ego_y += move_y
                                movement_occurred = True
                            else:
                                tello.move_right(int(move_y))
                                print(f"  Moved right {move_y} cm!")
                                time.sleep(0.5)
                                ego_y -= move_y
                                movement_occurred = True
                        else:
                            # Normal discrete movement
                            move_y = int(round(move_y))
                            if safe_dy > 0:
                                tello.move_left(move_y)
                                print(f"  Moved left {move_y} cm!")
                                time.sleep(0.5)
                                ego_y += move_y
                                movement_occurred = True
                            else:
                                tello.move_right(move_y)
                                print(f"  Moved right {move_y} cm!")
                                time.sleep(0.5)
                                ego_y -= move_y
                                movement_occurred = True
                    
                    if not movement_occurred:
                        print("  No movement - safe position too close to current position")
                
                # Update state history
                state_history.append((ego_x, ego_y, ego_z))
                
                # Print movement status
                if adversary_active and (abs(safe_x - desired_x) > 0.1 or abs(safe_y - desired_y) > 0.1):
                    # CBF already printed detailed info, just show summary
                    print(f"Moving towards goal {goal_index + 1}: ({target_x}, {target_y}), Current: ({ego_x:.1f}, {ego_y:.1f}) [CBF corrected trajectory]")
                else:
                    print(f"Moving towards goal {goal_index + 1}: ({target_x}, {target_y}), Current: ({ego_x:.1f}, {ego_y:.1f})")
                
                time.sleep(0.5)

            
            else:
                # Goal reached
                print(f"Goal {goal_index + 1} reached! ({target_x}, {target_y})")
                goal_index += 1
                time.sleep(1)
                
                if goal_index >= len(goals):
                    print("All goals processed! Landing the drone...")
                    if drone_in_air:
                        tello.land()
                        time.sleep(3)  # Delay after landing
                        state_history.append((ego_x, ego_y, ego_z))  # Record final state
                        print("Drone has landed.")
                        print(f"Final state: ({ego_x}, {ego_y}, {ego_z})")
                        print(f"Total states recorded: {len(state_history)}")
                    break




        # Landing control
        if key == ord('q'):
            if drone_in_air:
                tello.land()
                time.sleep(3)  # Delay after landing to allow drone to settle
                state_history.append((ego_x, ego_y, ego_z))  # Record final state before landing
                print("Drone has landed.")
                print(f"Final state: ({ego_x}, {ego_y}, {ego_z})")
                print(f"Total states recorded: {len(state_history)}")
            break



#####################################
# Landing the drone / finish program
#####################################
except Exception as e:
    print(f"Error occurred: {e}")
    if drone_in_air:
        print("Error occurred. Landing the drone.")
        tello.land()
        time.sleep(3)  # Delay after landing

except KeyboardInterrupt:
    print("Keyboard Interrupt received. Landing the drone.")
    if drone_in_air:
        tello.land()
        time.sleep(3)  # Delay after landing
    tello.streamoff()
    cv2.destroyAllWindows()

finally:
    tello.streamoff()
    cv2.destroyAllWindows()


Battery level: 100%
Press 't' to takeoff, 'q' to land and quit.
Drone is now airborne and hovering.
Drone moved up 30 cm. Current state: (0, 0, 20)
Pursuer set at position: (40, 0, 20)
Adversary detected at (40, 0), avoidance radius: 10 cm
  [CBF ACTIVE] Modified movement: desired (20.0, 0.0) -> safe (0.0, -20.0)
    Distance to adversary: 40.0 cm, Safety margin h: 1500.0
  Moved right 20 cm!
Moving towards goal 1: (40, -40), Current: (0.0, -20.0) [CBF corrected trajectory]
  [CBF ACTIVE] Modified movement: desired (20.0, -20.0) -> safe (4.0, -28.0)
    Distance to adversary: 44.7 cm, Safety margin h: 1900.0
  Using RC control: Moving forward 4.0 cm at 4 cm/s
  Using RC control: Moving right 8.0 cm at 4 cm/s
Moving towards goal 1: (40, -40), Current: (4.0, -28.0) [CBF corrected trajectory]
  [CBF ACTIVE] Modified movement: desired (24.0, -28.0) -> safe (11.5, -37.7)
    Distance to adversary: 45.6 cm, Safety margin h: 1980.0
  Using RC control: Moving forward 7.5 cm at 4 cm/s
  Using R